# 4.1 누락된 데이터 다루기 

### 테이블 형태 데이터에서 누락 값 식별

In [11]:
import pandas as pd
from io import StringIO

csv_data =\
'''
A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,
'''

df = pd.read_csv(StringIO(csv_data))

df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


#### 누락값 count - isnull.sum()
    : 열 마다 누락값 개수 count

In [13]:
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

### 누락값이 있는 훈련 샘플이나 특성 제외
    
    : 누락값이 있는 훈련 샘플의 행이나 열을 완전히 삭제 - dropna()
    
    누락된 데이터를 제거하면 안정적인 분석이 불가능 할 수 있음
    
    많은 특성 열 제거시 분류기가 클래스를 구분하는데 필요한 정보 손실 위험 증가

In [14]:
#axis = 0 : 행 삭제
df.dropna(axis=0)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [16]:
#axis = 1 :열 삭제
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


### 누락값 대체

    : 보간 기법을 사용해 데이터셋에 있는 다른 훈련 샘플로부터 누락된 값 대체
    
    1. 평균으로 대체 - 각 특성 열의 전체 평균으로 누락된 값 대체
    
    

##### 사이킷런의 SimplerImputer 클래스 사용, 각 특성 열에서 계산한 평균으로 변경

    : strategy에 mean, median, most_frequent, constant 으로 누락값 대체 

In [22]:
from sklearn.impute import SimpleImputer
import numpy as np

imr = SimpleImputer(missing_values=np.nan, strategy='mean')
imr = imr.fit(df.values)

imputed_data = imr.transform(df.values)
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

In [21]:
#strategy = constant, fill_value 값으로 대체값 설정 
from sklearn.impute import SimpleImputer
import numpy as np

imr = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)
imr = imr.fit(df.values)

imputed_data = imr.transform(df.values)
imputed_data

array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6.,  0.,  8.],
       [10., 11., 12.,  0.]])

##### 판다스  fillna( ) 

    : method 매개변수로 누락값 채울 수 있음. (axis =1 : 열 사용)
    
        - bfill : 누락값을 다음 행의 값으로 채움
        - ffill : 누락값을 이전 행의 값으로 채움

In [23]:
df.fillna(df.mean())

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.5,8.0
2,10.0,11.0,12.0,6.0


In [24]:
df.fillna(method='bfill')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,12.0,8.0
2,10.0,11.0,12.0,NaN


In [25]:
df.fillna(method='ffill')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,3.0,8.0
2,10.0,11.0,12.0,8.0


### 사이킷런 추정기 API

    SimplerImputer 클래스
    : 데이터 변환에 사용되는 사이킷런의 transformer 클래스
    
    fit 메서드 : 훈련데이터에서 모델 파라미터 학습
    transform 메서드: 학습한 파라미터로 데이터 변환

![SimpleImputer](https://thebook.io/img/080223/152.jpg)

# 4.2 범주형 데이터 다루기

 범주형 데이터 중 순서가 있어 정렬 가능한 데이터와 
 
 순서가 없어 정렬 불가능한 데이터 구분 필요

### 판다스를 사용한 범주형 데이터 인코딩

In [31]:
import pandas as pd

df= pd.DataFrame([
    ['green', 'M',10.1,'class1'],
    ['red', 'L',13.5,'class2'],
    ['blue', 'XL',15.3,'class1']])
    
df.columns = ['color','size','price','classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


#### 순서가 있는 특성 매핑

    : 학습 알고리즘의 순서특성 인식을 위해 범주형 문자열 값을 정수로 변경 해야함
    
     -> 딕셔너리로 매핑함수 직접 만들어야함

In [48]:
size_mapping = {
    'XL' : 3,
    'L' : 2,
    'M':1
}

df['size']=df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,0
1,red,2,13.5,1
2,blue,3,15.3,0


In [29]:
#정수값을 원래 문자열로 변경하고 싶다면 거꾸로 매핑하는 딕셔너리 정의

inv_size_mapping={v : k for k,v in size_mapping.items()}

df['size'].map(inv_size_mapping)

0     M
1     L
2    XL
Name: size, dtype: object

#### 순서가 있는 특성 인코딩
: 순서가 있는 특성의 범주 사이에서 수치적 크기에 대해 확신이 없거나, 
  
  두 범주 사이의 순서를 정의 할 수 없다면 임계값 사용해 인코딩 가능 
    
##### apply 메서드를 사용해 lamda 함수 적용

In [50]:
df['x>M']=df['size'].apply(lambda x: 1 if x in {'L','XL'} else 0)
df['x>L']=df['size'].apply(lambda x: 1 if x =='XL' else 0)
del df['size']
df

,color,price,classlabel,x>M,x>L
0,green,10.1,0,0,0
1,red,13.5,1,0,0
2,blue,15.3,0,0,0


### 클래스 레이블 인코딩
    : 클래스 레이블을 정수 배열로 전달

In [32]:
import numpy as np
class_mapping = {label:idx for idx,label in enumerate(np.unique(df['classlabel']))}

class_mapping

{'class1': 0, 'class2': 1}

In [33]:
df['classlabel']=df['classlabel'].map(class_mapping)
df

,color,size,price,classlabel
0,green,M,10.1,0
1,red,L,13.5,1
2,blue,XL,15.3,0


#### 사이킷런 LabelEncoder 클래스 사용

In [35]:
#fit_transform : fit 메서드와 transform 메서드 합친 단축 메서드
from sklearn.preprocessing import LabelEncoder
class_l = LabelEncoder()
y = class_l.fit_transform(df['classlabel'].values)
y

array([0, 1, 0], dtype=int64)

### 순서가 없는 특성에 One-Hot 인코딩 적용

color 특성은 순서가 없는 특성이므로 
blue = 0, green =1, red=2 으로 정수 인코딩 

-> 순서가 없는 특성에 순서가 부여됨

In [37]:
X = df[['color','size','price']].values

color_le =LabelEncoder()
X[:,0]=color_le.fit_transform(X[:,0])
X

array([[1, 'M', 10.1],
       [2, 'L', 13.5],
       [0, 'XL', 15.3]], dtype=object)

#### One-Hot 인코딩
: 순서 없는 특성에 들어 있는 고유값 마다 새로운 dummy 특성 만들기 

-> color 특성을 blue, green, red 3 개의 새로운 특성으로 만들기

##### preprocessing 모듈의 OneHotEncoder 사용

In [39]:
from sklearn.preprocessing import OneHotEncoder

X = df[['color','size','price']].values

color_ohe = OneHotEncoder()
#color 열에 대해서만 one-hot 인코딩
color_ohe.fit_transform(X[:,0].reshape(-1,1)).toarray()

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

##### Pandas의 get_dummies() 메서드

In [45]:
pd.get_dummies(df[['price','color','size']], columns=['color'])

,price,size,color_blue,color_green,color_red
0,10.1,M,0,1,0
1,13.5,L,0,0,1
2,15.3,XL,1,0,0


##### One-Hot 인코딩과 다중공선성
    : 특성간의 상관관계가 높으면 역행렬 계산이 어려워 수치적으로 불안정해짐
        
    -> 특성 열 하나를 삭제해 변수 값 상관관계 감소

##### drop_first = True

In [49]:
pd.get_dummies(df[['price','color','size']],drop_first=True)

,price,size,color_green,color_red
0,10.1,1,1,0
1,13.5,2,0,1
2,15.3,3,0,0
